In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Read JSON file in chunks
chunk_size = 1000000  # Adjust the chunk size as needed
chunks = []
with open('Kindle_Store.json', 'r') as file:
    for chunk in pd.read_json(file, chunksize=chunk_size, lines=True):
        chunks.append(chunk)

# Concatenate all chunks to form a single DataFrame
df = pd.concat(chunks, ignore_index=True)


In [3]:

df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"12 29, 2012",A27UD5HYAKBL97,1423600150,{'Format:': ' Hardcover'},Cheryl,If you like making salsas this is a great book...,Great Book,1356739200,NaN,NaN
1,5,True,"03 6, 2012",A8P5DK8LLOYGH,1423600150,{'Format:': ' Hardcover'},Shay365,great little book. simple and right to the poi...,great little book,1330992000,3,NaN
2,5,True,"08 12, 2009",A3OM9W7DXSUIIY,1423600150,{'Format:': ' Hardcover'},R. Peckham,This book has good pics of the recipes and eas...,very good bok with good ideas.,1250035200,NaN,NaN
3,3,False,"11 3, 2006",A3DPAR2PWB9BT8,1423600150,{'Format:': ' Hardcover'},JJSS,"I admire the SF School of Cooking, but was dis...",Ok cookbook,1162512000,3,NaN
4,5,False,"04 11, 2006",A2ONTBSAX7GIOV,1423600150,{'Format:': ' Hardcover'},Three Guys from Miami,Many people are surprised to find out that we ...,Fresh and Tasty Ideas from Santa Fe!,1144713600,15,NaN


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# Assuming 'df' is your existing DataFrame
chunk_size = 1000  # Adjust the chunk size as needed
text_data_chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Function for text preprocessing
def preprocess_text(text_series):
    text_series = text_series.str.lower()  # Lowercasing
    text_series = text_series.str.replace('[{}]'.format(string.punctuation), '')  # Removing punctuation
    stop_words = set(stopwords.words('english'))
    
    def process_text(text):
        tokens = word_tokenize(text)  # Tokenization
        tokens = [word for word in tokens if word not in stop_words]  # Removing stopwords
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]  # Stemming
        return ' '.join(tokens)
    
    processed_text = text_series.apply(process_text)
    return processed_text

# Process text in chunks and save processed chunks to disk
for i, chunk in enumerate(text_data_chunks):
    processed_chunk = preprocess_text(chunk['reviewText'])
    chunk_vectorizer = CountVectorizer()
    chunk_bag_of_words = chunk_vectorizer.fit_transform(processed_chunk)

    # Convert bag-of-words to DataFrame
    chunk_bow_df = pd.DataFrame(chunk_bag_of_words.toarray(), columns=chunk_vectorizer.get_feature_names_out())

    # Save processed chunk to disk (for example, save as CSV)
    chunk_bow_df.to_csv(f'processed_chunk_{i}.csv', index=False)


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
bow_df.head()